In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('potato.csv')
df

,Date,p/kg
0,01-Sep,63
1,02-Sep,66
2,03-Sep,68
3,04-Sep,70
4,05-Sep,70
...,...,...
103,13-Dec,36
104,14-Dec,36
105,15-Dec,36
106,16-Dec,34


In [4]:
first_label=(df.loc[1][1]-df.loc[0][1])/df.loc[0][1]
first_label

0.047619047619047616

In [14]:
labels=df['p/kg']
temp=[]
for i in range(1,len(labels)):
    result=(labels[i]-labels[i-1])/labels[i-1]
    label=-2
    if result>0:
        label=1
    elif result==0:
        label=0
    else:
        label=-1
    temp.append(label)
new_temp=[0]
new_temp = new_temp+temp
new_temp

[0,
 1,
 1,
 1,
 0,
 -1,
 -1,
 0,
 -1,
 0,
 1,
 0,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 -1,
 0,
 -1,
 0,
 0,
 1,
 1,
 0,
 0,
 -1,
 0,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 0,
 -1,
 0,
 1,
 0,
 -1,
 0,
 1,
 0,
 1,
 1,
 1,
 -1,
 1,
 0,
 -1,
 0,
 -1,
 1,
 1,
 0,
 0,
 1,
 -1,
 1,
 -1,
 0,
 -1,
 0,
 1,
 1,
 0,
 -1,
 0,
 -1,
 0,
 1,
 -1,
 1,
 -1,
 0,
 1,
 0,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 0,
 -1,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 0,
 0,
 -1,
 0]

In [15]:
df['labels']=new_temp
df

In [19]:
x_train,y_train =df['p/kg'],df['labels']

In [27]:
x_train
y_train

0      0
1      1
2      1
3      1
4      0
      ..
103    0
104    0
105    0
106   -1
107    0
Name: labels, Length: 108, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split
training_data,testing_data=train_test_split(df,test_size=0.2,random_state=25)
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 86
No. of testing examples: 22


In [36]:
training_data

,Date,p/kg,labels
94,04-Dec,56,-1
97,07-Dec,55,0
74,14-Nov,68,0
63,03-Nov,70,-1
0,01-Sep,63,0
...,...,...,...
61,01-Nov,71,-1
15,16-Sep,63,-1
90,30-Nov,69,1
62,02-Nov,71,0
